In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd

In [2]:
from os import walk
import os
mypath = os.path.abspath(os.getcwd()) + "/filtered_genomes"

files = []
for (dirpath, dirnames, filenames) in walk(mypath):
    files.extend(filenames)
    break

ext = "embl"

In [3]:
FileNames = []
GCcontent = []
IntergenomicGC = []
GC3content = []
PhePer = []
Leu4Per = []
Leu2Per = []
IlePer = []
MetPer = []
ValPer = []
Ser4Per = []
Ser2Per = []
ProPer = []
ThrPer = []
AlaPer = []
TyrPer = []
HisPer = []
GlnPer = []
AsnPer = []
LysPer = []
AspPer = []
GluPer = []
CysPer = []
TrpPer = []
Arg2Per = []
Arg4Per = []
GlyPer = []
PheGC = []
ValGC = []
Ser4GC =[]
Ser2GC =[]
ProGC = []
ThrGC = []
AlaGC = []
TyrGC = []
HisGC = []
GlnGC = []
AsnGC = []
LysGC = []
AspGC = []
GluGC = []
CysGC = []
Arg4GC = []
Arg2GC = []
Leu4GC = []
Leu2GC = []
GlyGC = []

files_tot = len(files)
for index, name in enumerate(files):
    print("\n")
    print(f"Analyzing file {index+1} of {files_tot}: {name}")
    print("\n")
    record = SeqIO.read(f'filtered_genomes/{name}', ext)
    sequence = record.seq
    FileNames.append(str(name))
    
    start = 0
    coding_sequences = []
    intergenomic_sequence = []  # It will contain the sequences out of the locations bounds
    intergenomic_sequence_list = []  # It will contain the locations bounds
    intergenomic_sequence_string = ""  # intergenomic sequence
    
    for feature in record.features:

        locations= feature.location
        s = int(locations.nofuzzy_start)
        e = int(locations.nofuzzy_end)
        st = int(locations.strand)
        
        gene_start = int(locations.nofuzzy_start)

        if gene_start > start:
            ig_sequence = sequence[start:gene_start]
            intergenomic_sequence.append(str(ig_sequence))
            intergenomic_sequence_list.append((start, gene_start))

        new_start = int(locations.nofuzzy_end)

        if new_start != len(sequence):
            start = new_start

        #Reversing complementary strands
        if st == -1:
            sequence_negativestrand = str(sequence[s:e])
            sequence_negativestrand= sequence_negativestrand[::-1]
            sequence_negativestrand= sequence_negativestrand.replace("A","b")
            sequence_negativestrand= sequence_negativestrand.replace("T","A")
            sequence_negativestrand= sequence_negativestrand.replace("b","T")
            sequence_negativestrand= sequence_negativestrand.replace("C","d")
            sequence_negativestrand= sequence_negativestrand.replace("G","C")
            sequence_negativestrand= sequence_negativestrand.replace("d","G")
            
            genes = str(sequence_negativestrand)

        #Discarding the pseudogenes

        ok = 0
        if len(genes)%3 != 0:
            ok = 1
        elif genes[:3] != "ATG" and genes[:3] != "GTG" and genes[:3] != "TTG":
            ok = 2
        elif genes[-3:] != "TAA" and genes[-3:] != "TAG" and genes[-3:] != "TGA":
            ok = 3
        elif len(genes) == len(sequence):
            ok = 4
        check5 = False
        if check5:
            for i in range(0, len(genes)-3, 3): # Splitting the sequence in codons discarding the last one
                codon = genes[i: i + 3]
                if codon in ["TAA", "TAG", "TGA"]:
                    ok = 5  # Raises an error if a codon in the middle of the gene is a stop codon

        if ok == 0:
            coding_sequences.append(genes)

        #checking for errors
        check = False
        if check and st == 1 and ok != 0:
            print(locations)
            print(ok)
            print(genes[:3])
            print(genes[-3:])
        if check5 and ok == 5:
            print("Stop codon error:")
            print(f"CDS:\n{genes}")
            
    if start != len(sequence):
        ig_sequence = sequence[start:len(sequence)]
        intergenomic_sequence.append(str(ig_sequence))
        intergenomic_sequence_list.append((start, gene_start))

    intergenomic_sequence_string = "".join(intergenomic_sequence)

    # Genomic GC content
    GC = record.seq.count("G") + record.seq.count("C")
    GC_content = (GC / len(record.seq)) * 100
    print(f"Genomic GC content:  {GC_content:.2f}%")
    GCcontent.append(GC_content)

    # Intergenomic GC content
    IgGC = intergenomic_sequence_string.count("G") + intergenomic_sequence_string.count("C")
    IgGC_content = (IgGC / len(intergenomic_sequence_string)) * 100
    print(f"Intergenomic GC content: {IgGC_content:.2f}%")
    IntergenomicGC.append(IgGC_content)
    
    #GC3 content
    Gcount = 0
    Ccount = 0
    Acount = 0
    Tcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            # while codons[:-3] != "TAA" and codons[:-3] !="TAG" and codons[:-3] !="TGA":
            # Does it get rid of only that codon though?
            G3 = codons[2].count("G")
            Gcount += G3
            C3 = codons[2].count("C")
            Ccount += C3
            A3 = codons[2].count("A")
            Acount += A3
            T3 = codons[2].count("T")
            Tcount += T3
    GC3 = (Gcount + Ccount) / (Gcount + Ccount + Acount + Tcount) * 100
    print(f'GC3: {(GC3):.2f}%')
    GC3content.append(GC3)
          
    # Amino acids usage
    coding_sequencewhole = "".join(coding_sequences)
    aminoacids_sequence = Seq(coding_sequencewhole).translate()
    # number of Valine
    print(f'Percentage of Valine:        {(aminoacids_sequence.count("V") / len(aminoacids_sequence) * 100):.2f}%')
    ValPer.append(aminoacids_sequence.count("V") / len(aminoacids_sequence) * 100)
    # number of Phenilalanine
    print(f'Percentage of Phenilalanine: {(aminoacids_sequence.count("F") / len(aminoacids_sequence) * 100):.2f}%')
    PhePer.append(aminoacids_sequence.count("F") / len(aminoacids_sequence) * 100)
    # number of Glycine
    print(f'Percentage of Glycine:       {(aminoacids_sequence.count("G") / len(aminoacids_sequence) * 100):.2f}%')
    GlyPer.append(aminoacids_sequence.count("G") / len(aminoacids_sequence) * 100)
    # number of Isoleucine
    print(f'Percentage of Isoleucine:    {(aminoacids_sequence.count("I") / len(aminoacids_sequence) * 100):.2f}%')
    IlePer.append(aminoacids_sequence.count("I") / len(aminoacids_sequence) * 100)
    # number of Proline
    print(f'Percentage of Proline:       {(aminoacids_sequence.count("P") / len(aminoacids_sequence) * 100):.2f}%')
    ProPer.append(aminoacids_sequence.count("P") / len(aminoacids_sequence) * 100)
    # number of Threonine
    print(f'Percentage of Threonine:     {(aminoacids_sequence.count("T") / len(aminoacids_sequence) * 100):.2f}%')
    ThrPer.append(aminoacids_sequence.count("T") / len(aminoacids_sequence) * 100)
    # number of Alanine
    print(f'Percentage of Alanine:       {(aminoacids_sequence.count("A") / len(aminoacids_sequence) * 100):.2f}%')
    AlaPer.append(aminoacids_sequence.count("A") / len(aminoacids_sequence) * 100)
    # number of Tyrosine
    print(f'Percentage of Tyrosine:      {(aminoacids_sequence.count("Y") / len(aminoacids_sequence) * 100):.2f}%')
    TyrPer.append(aminoacids_sequence.count("Y") / len(aminoacids_sequence) * 100)
    # number of Histidine
    print(f'Percentage of Histidine:     {(aminoacids_sequence.count("H") / len(aminoacids_sequence) * 100):.2f}%')
    HisPer.append(aminoacids_sequence.count("H") / len(aminoacids_sequence) * 100)
    # number of Glutamine
    print(f'Percentage of Glutamine:     {(aminoacids_sequence.count("Q") / len(aminoacids_sequence) * 100):.2f}%')
    GlnPer.append(aminoacids_sequence.count("Q") / len(aminoacids_sequence) * 100)
    # number of Asparagine
    print(f'Percentage of Asparagine:    {(aminoacids_sequence.count("N") / len(aminoacids_sequence) * 100):.2f}%')
    AsnPer.append(aminoacids_sequence.count("N") / len(aminoacids_sequence) * 100)
    # number of Lysine
    print(f'Percentage of Lysine:        {(aminoacids_sequence.count("K") / len(aminoacids_sequence) * 100):.2f}%')
    LysPer.append(aminoacids_sequence.count("K") / len(aminoacids_sequence) * 100)
    # number of Aspartic Acid
    print(f'Percentage of Aspartic Acid: {(aminoacids_sequence.count("D") / len(aminoacids_sequence) * 100):.2f}%')
    AspPer.append(aminoacids_sequence.count("D") / len(aminoacids_sequence) * 100)
    # number of Glutamic Acid
    print(f'Percentage of Glutamic Acid: {(aminoacids_sequence.count("E") / len(aminoacids_sequence) * 100):.2f}%')
    GluPer.append(aminoacids_sequence.count("E") / len(aminoacids_sequence) * 100)
    # number of Cysteine
    print(f'Percentage of Cysteine:      {(aminoacids_sequence.count("C") / len(aminoacids_sequence) * 100):.2f}%')
    CysPer.append(aminoacids_sequence.count("C") / len(aminoacids_sequence) * 100)
    # number of Tryptophan
    print(f'Percentage of Tryptophan     {(aminoacids_sequence.count("W") / len(aminoacids_sequence) * 100):.2f}%')
    TrpPer.append(aminoacids_sequence.count("W") / len(aminoacids_sequence) * 100)
    # number of Methionine
    aminoacids_sequence = Seq(coding_sequencewhole[3:]).translate()
    print(f'Percentage of Methionine:    {(aminoacids_sequence.count("M") / len(aminoacids_sequence) * 100):.2f}%')
    MetPer.append(aminoacids_sequence.count("M") / len(aminoacids_sequence) * 100)
        
    # Codon usage for Arginine + R4 vs R2
    CGAcount = 0
    CGCcount = 0
    CGGcount = 0
    CGTcount = 0
    AGAcount = 0
    AGGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            CGA = codons.count("CGA")
            CGAcount += CGA
            CGC = codons.count("CGC")
            CGCcount += CGC
            CGG = codons.count("CGG")
            CGGcount += CGG
            CGT = codons.count("CGT")
            CGTcount += CGT
            AGA = codons.count("AGA")
            AGAcount += AGA
            AGG = codons.count("AGG")
            AGGcount += AGG
    Arginine4Total = CGGcount + CGAcount + CGCcount + CGTcount
    Arginine2Total = AGAcount + AGGcount
    Arg4Per.append(Arginine4Total)
    Arg2Per.append(Arginine2Total)
    Arg4GC.append((CGGcount + CGCcount) / (Arginine4Total) * 100)
    Arg2GC.append(AGGcount / (Arginine2Total) * 100)
    
    #Codon usage for Leucine + L4 vs L2
    CTAcount = 0
    CTCcount = 0
    CTGcount = 0
    CTTcount = 0
    TTAcount = 0
    TTGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            CTA = codons.count("CTA")
            CTAcount += CTA
            CTC = codons.count("CTC")
            CTCcount += CTC
            CTG = codons.count("CTG")
            CTGcount += CTG
            CTT = codons.count("CTT")
            CTTcount += CTT
            TTA = codons.count("TTA")
            TTAcount += TTA
            TTG = codons.count("TTG")
            TTGcount += TTG
    Leucine4Total = CTGcount + CTAcount + CTCcount + CTTcount
    Leucine2Total = TTAcount + TTGcount
    Leu4Per.append(Leucine4Total)
    Leu2Per.append(Leucine2Total)
    Leu4GC.append((CTGcount + CTCcount) / (Leucine4Total) * 100)
    Leu2GC.append(TTGcount / (Leucine2Total) * 100)
    
    #Codon usage for Serine + S4 vs S2
    TCAcount = 0
    TCCcount = 0
    TCGcount = 0
    TCTcount = 0
    AGTcount = 0
    AGCcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            TCA = codons.count("TCA")
            TCAcount += TCA
            TCC = codons.count("TCC")
            TCCcount += TCC
            TCG = codons.count("TCG")
            TCGcount += TCG
            TCT = codons.count("TCT")
            TCTcount += TCT
            AGT = codons.count("AGT")
            AGTcount += AGT
            AGC = codons.count("AGC")
            AGCcount += AGC
    Serine4Total = TCGcount + TCAcount + TCCcount + TCTcount
    Serine2Total = AGTcount + AGCcount
    Ser4Per.append(Serine4Total)
    Ser2Per.append(Serine2Total)
    Ser4GC.append((TCGcount + TCCcount) / (Serine4Total) * 100)
    Ser2GC.append(AGCcount / (Serine2Total) * 100)

    # Codon usage for Valine
    GTAcount = 0
    GTCcount = 0
    GTGcount = 0
    GTTcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            GTA = codons.count("GTA")
            GTAcount += GTA
            GTC = codons.count("GTC")
            GTCcount += GTC
            GTG = codons.count("GTG")
            GTGcount += GTG
            GTT = codons.count("GTT")
            GTTcount += GTT
    ValineTotal = GTAcount + GTCcount + GTGcount + GTTcount
    ValGC.append((GTGcount + GTCcount) / (ValineTotal) * 100)

    # Codon usage for Proline
    CCAcount = 0
    CCCcount = 0
    CCGcount = 0
    CCTcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            CCA = codons.count("CCA")
            CCAcount += CCA
            CCC = codons.count("CCC")
            CCCcount += CCC
            CCG = codons.count("CCG")
            CCGcount += CCG
            CCT = codons.count("CCT")
            CCTcount += CCT
    ProlineTotal = CCAcount + CCCcount + CCGcount + CCTcount
    ProGC.append((CCCcount + CCGcount) * 100 / (ProlineTotal))

    # Codon usage for Threonine
    ACAcount = 0
    ACCcount = 0
    ACGcount = 0
    ACTcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            ACA = codons.count("ACA")
            ACAcount += ACA
            ACC = codons.count("ACC")
            ACCcount += ACC
            ACG = codons.count("ACG")
            ACGcount += ACG
            ACT = codons.count("ACT")
            ACTcount += ACT
    ThreonineTotal = ACAcount + ACCcount + ACGcount + ACTcount
    ThrGC.append((ACCcount + ACGcount)* 100 / (ThreonineTotal))

    # Codon usage for Alanine
    GCAcount = 0
    GCCcount = 0
    GCGcount = 0
    GCTcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            GCA = codons.count("GCA")
            GCAcount += GCA
            GCC = codons.count("GCC")
            GCCcount += GCC
            GCG = codons.count("GCG")
            GCGcount += GCG
            GCT = codons.count("GCT")
            GCTcount += GCT
    AlanineTotal = GCAcount + GCCcount + GCGcount + GCTcount
    AlaGC.append((GCGcount + GCCcount) * 100 / (AlanineTotal))
    
    # Codon usage for Phenylalanine
    TTTcount = 0
    TTCcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            TTT = codons.count("TTT")
            TTTcount += TTT
            TTC = codons.count("TTC")
            TTCcount += TTC
    PhenylalanineTotal = TTTcount + TTCcount
    PheGC.append(TTCcount * 100 / PhenylalanineTotal)
    
    # Codon usage for Tyrosine
    TATcount = 0
    TACcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            TAT = codons.count("TAT")
            TATcount += TAT
            TAC = codons.count("TAC")
            TACcount += TAC
    TyrosineTotal = TATcount + TACcount
    TyrGC.append(TACcount * 100 / TyrosineTotal)
    
    # Codon usage for Histidine
    CATcount = 0
    CACcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            CAT = codons.count("CAT")
            CATcount += CAT
            CAC = codons.count("CAC")
            CACcount += CAC
            HistidineTotal = CATcount + CACcount
    HisGC.append(CACcount * 100 / HistidineTotal)
    
    # Codon usage for Glutamine
    CAAcount = 0
    CAGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            CAA = codons.count("CAA")
            CAAcount += CAA
            CAG = codons.count("CAG")
            CAGcount += CAG
    GlutamineTotal = CAAcount + CAGcount
    GlnGC.append(CAGcount * 100 / GlutamineTotal)    
    
    # Codon usage for Asparagine
    AATcount = 0
    AACcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            AAT = codons.count("AAT")
            AATcount += AAT
            AAC = codons.count("AAC")
            AACcount += AAC
    AsparagineTotal = AATcount + AACcount
    AsnGC.append(AACcount * 100 / AsparagineTotal)
    
    # Codon usage for Lysine
    AAAcount = 0
    AAGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            AAA = codons.count("AAA")
            AAAcount += AAA
            AAG = codons.count("AAG")
            AAGcount += AAG
    LysineTotal = AAAcount + AAGcount
    LysGC.append(AAGcount * 100 / LysineTotal)
    
    # Codon usage for Aspartic Acid
    GATcount = 0
    GACcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            GAT = codons.count("GAT")
            GATcount += GAT
            GAC = codons.count("GAC")
            GACcount += GAC
    AsparticTotal = GATcount + GACcount
    AspGC.append(GACcount * 100 / AsparticTotal)
    
    # Codon usage for Glutamic Acid
    GAAcount = 0
    GAGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            GAA = codons.count("GAA")
            GAAcount += GAA
            GAG = codons.count("GAG")
            GAGcount += GAG
    GlutamicTotal = GAAcount + GAGcount
    GluGC.append(GAGcount * 100 / GlutamicTotal)
    
    # Codon usage for Cysteine
    TGTcount = 0
    TGCcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            TGT = codons.count("TGT")
            TGTcount += TGT
            TGC = codons.count("TGC")
            TGCcount += TGC
    CysteineTotal = TGTcount + TGCcount
    CysGC.append(TGCcount * 100 / CysteineTotal)
    
    # Codon usage for Glycine
    GGTcount = 0
    GGCcount = 0
    GGAcount = 0
    GGGcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            GGA = codons.count("GGA")
            GGAcount += GGA
            GGC = codons.count("GGC")
            GGCcount += GGC
            GGG = codons.count("GGG")
            GGGcount += GGG
            GGT = codons.count("GGT")
            GGTcount += GGT
    GlycineTotal = GGAcount + GGCcount + GGGcount + GGTcount
    GlyGC.append((GGGcount + GGCcount) * 100 / (GlycineTotal))
    
    



Analyzing file 1 of 645: AE000657.embl


Genomic GC content:  43.48%
Intergenomic GC content: 37.10%
GC3: 48.08%
Percentage of Valine:        7.94%
Percentage of Phenilalanine: 5.13%
Percentage of Glycine:       6.74%
Percentage of Isoleucine:    7.31%
Percentage of Proline:       4.06%
Percentage of Threonine:     4.20%
Percentage of Alanine:       5.88%
Percentage of Tyrosine:      4.12%
Percentage of Histidine:     1.55%
Percentage of Glutamine:     2.04%
Percentage of Asparagine:    3.59%
Percentage of Lysine:        9.38%
Percentage of Aspartic Acid: 4.30%
Percentage of Glutamic Acid: 9.61%
Percentage of Cysteine:      0.79%
Percentage of Tryptophan     0.93%
Percentage of Methionine:    1.87%


Analyzing file 2 of 645: AE014184.embl


Genomic GC content:  46.33%
Intergenomic GC content: 44.71%
GC3: 42.48%
Percentage of Valine:        8.02%
Percentage of Phenilalanine: 4.20%
Percentage of Glycine:       7.34%
Percentage of Isoleucine:    6.71%
Percentage of Proline:       4.55%


Percentage of Methionine:    2.12%


Analyzing file 13 of 645: AM711867.embl


Genomic GC content:  72.66%
Intergenomic GC content: 71.74%
GC3: 95.44%
Percentage of Valine:        9.41%
Percentage of Phenilalanine: 2.77%
Percentage of Glycine:       9.27%
Percentage of Isoleucine:    4.10%
Percentage of Proline:       5.73%
Percentage of Threonine:     6.06%
Percentage of Alanine:       14.19%
Percentage of Tyrosine:      1.82%
Percentage of Histidine:     1.97%
Percentage of Glutamine:     2.63%
Percentage of Asparagine:    1.53%
Percentage of Lysine:        1.69%
Percentage of Aspartic Acid: 6.57%
Percentage of Glutamic Acid: 5.29%
Percentage of Cysteine:      0.46%
Percentage of Tryptophan     1.37%
Percentage of Methionine:    1.60%


Analyzing file 14 of 645: AM743169.embl


Genomic GC content:  66.32%
Intergenomic GC content: 66.18%
GC3: 84.03%
Percentage of Valine:        7.38%
Percentage of Phenilalanine: 3.25%
Percentage of Glycine:       8.60%
Percentage of Isoleucine:    4.0

Percentage of Methionine:    2.67%


Analyzing file 25 of 645: AP009152.embl


Genomic GC content:  71.16%
Intergenomic GC content: 71.90%
GC3: 93.19%
Percentage of Valine:        9.18%
Percentage of Phenilalanine: 2.86%
Percentage of Glycine:       9.03%
Percentage of Isoleucine:    3.44%
Percentage of Proline:       5.75%
Percentage of Threonine:     6.10%
Percentage of Alanine:       12.76%
Percentage of Tyrosine:      1.94%
Percentage of Histidine:     2.43%
Percentage of Glutamine:     3.20%
Percentage of Asparagine:    2.00%
Percentage of Lysine:        2.10%
Percentage of Aspartic Acid: 5.84%
Percentage of Glutamic Acid: 6.01%
Percentage of Cysteine:      0.70%
Percentage of Tryptophan     1.46%
Percentage of Methionine:    1.94%


Analyzing file 26 of 645: AP009153.embl


Genomic GC content:  64.27%
Intergenomic GC content: 61.58%
GC3: 77.00%
Percentage of Valine:        8.40%
Percentage of Phenilalanine: 3.22%
Percentage of Glycine:       8.44%
Percentage of Isoleucine:    4.2

Percentage of Methionine:    2.32%


Analyzing file 37 of 645: AP010953.embl


Genomic GC content:  50.68%
Intergenomic GC content: 42.04%
GC3: 55.54%
Percentage of Valine:        6.95%
Percentage of Phenilalanine: 3.75%
Percentage of Glycine:       7.28%
Percentage of Isoleucine:    5.86%
Percentage of Proline:       4.40%
Percentage of Threonine:     5.46%
Percentage of Alanine:       9.47%
Percentage of Tyrosine:      2.87%
Percentage of Histidine:     2.22%
Percentage of Glutamine:     4.43%
Percentage of Asparagine:    3.99%
Percentage of Lysine:        4.50%
Percentage of Aspartic Acid: 5.20%
Percentage of Glutamic Acid: 5.88%
Percentage of Cysteine:      1.17%
Percentage of Tryptophan     1.53%
Percentage of Methionine:    2.73%


Analyzing file 38 of 645: AP010968.embl


Genomic GC content:  74.20%
Intergenomic GC content: 73.08%
GC3: 95.96%
Percentage of Valine:        8.20%
Percentage of Phenilalanine: 2.54%
Percentage of Glycine:       9.70%
Percentage of Isoleucine:    2.59

Percentage of Methionine:    2.40%


Analyzing file 49 of 645: AP012051.embl


Genomic GC content:  35.37%
Intergenomic GC content: 31.30%
GC3: 27.47%
Percentage of Valine:        7.33%
Percentage of Phenilalanine: 5.49%
Percentage of Glycine:       6.63%
Percentage of Isoleucine:    9.29%
Percentage of Proline:       3.73%
Percentage of Threonine:     5.23%
Percentage of Alanine:       5.62%
Percentage of Tyrosine:      3.87%
Percentage of Histidine:     1.43%
Percentage of Glutamine:     2.11%
Percentage of Asparagine:    5.02%
Percentage of Lysine:        8.43%
Percentage of Aspartic Acid: 4.91%
Percentage of Glutamic Acid: 7.48%
Percentage of Cysteine:      0.59%
Percentage of Tryptophan     0.68%
Percentage of Methionine:    1.93%


Analyzing file 50 of 645: AP012052.embl


Genomic GC content:  70.28%
Intergenomic GC content: 70.36%
GC3: 90.16%
Percentage of Valine:        9.28%
Percentage of Phenilalanine: 3.11%
Percentage of Glycine:       8.94%
Percentage of Isoleucine:    4.25

...

Percentage of Methionine:    2.82%


Analyzing file 588 of 645: CP010848.embl


Genomic GC content:  61.48%
Intergenomic GC content: 58.96%
GC3: 70.39%
Percentage of Valine:        9.03%
Percentage of Phenilalanine: 3.13%
Percentage of Glycine:       8.47%
Percentage of Isoleucine:    4.83%
Percentage of Proline:       5.05%
Percentage of Threonine:     6.15%
Percentage of Alanine:       12.20%
Percentage of Tyrosine:      2.05%
Percentage of Histidine:     2.12%
Percentage of Glutamine:     2.92%
Percentage of Asparagine:    2.26%
Percentage of Lysine:        2.30%
Percentage of Aspartic Acid: 5.75%
Percentage of Glutamic Acid: 5.75%
Percentage of Cysteine:      0.66%
Percentage of Tryptophan     1.31%
Percentage of Methionine:    1.56%


Analyzing file 589 of 645: CP010954.embl


Genomic GC content:  64.79%
Intergenomic GC content: 59.39%
GC3: 82.10%
Percentage of Valine:        6.99%
Percentage of Phenilalanine: 3.54%
Percentage of Glycine:       9.03%
Percentage of Isoleucine:    5

Percentage of Methionine:    2.33%


Analyzing file 600 of 645: CU207211.embl


Genomic GC content:  54.29%
Intergenomic GC content: 46.67%
GC3: 63.06%
Percentage of Valine:        7.11%
Percentage of Phenilalanine: 3.81%
Percentage of Glycine:       7.42%
Percentage of Isoleucine:    5.87%
Percentage of Proline:       4.50%
Percentage of Threonine:     5.39%
Percentage of Alanine:       11.16%
Percentage of Tyrosine:      2.51%
Percentage of Histidine:     2.26%
Percentage of Glutamine:     4.03%
Percentage of Asparagine:    3.40%
Percentage of Lysine:        4.48%
Percentage of Aspartic Acid: 5.24%
Percentage of Glutamic Acid: 5.51%
Percentage of Cysteine:      0.93%
Percentage of Tryptophan     1.27%
Percentage of Methionine:    2.71%


Analyzing file 601 of 645: CU207366.embl


Genomic GC content:  36.61%
Intergenomic GC content: 29.39%
GC3: 31.37%
Percentage of Valine:        5.94%
Percentage of Phenilalanine: 5.23%
Percentage of Glycine:       6.51%
Percentage of Isoleucine:    7

Percentage of Methionine:    2.68%


Analyzing file 612 of 645: FP475956.embl


Genomic GC content:  63.67%
Intergenomic GC content: 57.74%
GC3: 79.47%
Percentage of Valine:        7.22%
Percentage of Phenilalanine: 3.42%
Percentage of Glycine:       7.92%
Percentage of Isoleucine:    4.34%
Percentage of Proline:       5.52%
Percentage of Threonine:     5.08%
Percentage of Alanine:       12.89%
Percentage of Tyrosine:      2.23%
Percentage of Histidine:     2.43%
Percentage of Glutamine:     4.77%
Percentage of Asparagine:    2.53%
Percentage of Lysine:        3.02%
Percentage of Aspartic Acid: 5.14%
Percentage of Glutamic Acid: 4.81%
Percentage of Cysteine:      0.92%
Percentage of Tryptophan     1.57%
Percentage of Methionine:    2.33%


Analyzing file 613 of 645: FP476056.embl


Genomic GC content:  42.78%
Intergenomic GC content: 34.61%
GC3: 46.75%
Percentage of Valine:        6.32%
Percentage of Phenilalanine: 5.01%
Percentage of Glycine:       7.08%
Percentage of Isoleucine:    6

Percentage of Methionine:    3.25%


Analyzing file 624 of 645: FP929062.embl


Genomic GC content:  46.35%
Intergenomic GC content: 44.61%
GC3: 57.36%
Percentage of Valine:        6.93%
Percentage of Phenilalanine: 4.05%
Percentage of Glycine:       7.55%
Percentage of Isoleucine:    6.85%
Percentage of Proline:       3.48%
Percentage of Threonine:     5.54%
Percentage of Alanine:       8.14%
Percentage of Tyrosine:      3.85%
Percentage of Histidine:     1.75%
Percentage of Glutamine:     2.94%
Percentage of Asparagine:    4.09%
Percentage of Lysine:        6.73%
Percentage of Aspartic Acid: 5.64%
Percentage of Glutamic Acid: 7.52%
Percentage of Cysteine:      1.46%
Percentage of Tryptophan     0.92%
Percentage of Methionine:    3.23%


Analyzing file 625 of 645: FQ312005.embl


Genomic GC content:  36.75%
Intergenomic GC content: 36.15%
GC3: 30.86%
Percentage of Valine:        5.95%
Percentage of Phenilalanine: 5.26%
Percentage of Glycine:       6.03%
Percentage of Isoleucine:    7.

Percentage of Methionine:    2.62%


Analyzing file 636 of 645: HF680312.embl


Genomic GC content:  46.63%
Intergenomic GC content: 43.86%
GC3: 44.66%
Percentage of Valine:        6.90%
Percentage of Phenilalanine: 3.79%
Percentage of Glycine:       6.95%
Percentage of Isoleucine:    6.00%
Percentage of Proline:       4.09%
Percentage of Threonine:     5.46%
Percentage of Alanine:       9.46%
Percentage of Tyrosine:      2.88%
Percentage of Histidine:     2.20%
Percentage of Glutamine:     4.35%
Percentage of Asparagine:    4.00%
Percentage of Lysine:        4.37%
Percentage of Aspartic Acid: 5.96%
Percentage of Glutamic Acid: 6.10%
Percentage of Cysteine:      0.95%
Percentage of Tryptophan     1.36%
Percentage of Methionine:    2.55%


Analyzing file 637 of 645: HF951689.embl


Genomic GC content:  54.41%
Intergenomic GC content: 48.35%
GC3: 59.62%
Percentage of Valine:        7.15%
Percentage of Phenilalanine: 3.68%
Percentage of Glycine:       7.50%
Percentage of Isoleucine:    5.

In [4]:
df = pd.DataFrame(
    data=list(zip(
        FileNames,
        GCcontent,
        IntergenomicGC,
        GC3content,
        PhePer,
        Leu2Per,
        Leu4Per,
        IlePer,
        MetPer,
        ValPer,
        Ser2Per,
        Ser4Per,
        ProPer,
        ThrPer,
        AlaPer,
        TyrPer,
        HisPer,
        GlnPer,
        AsnPer,
        LysPer,
        AspPer,
        GluPer,
        CysPer,
        TrpPer,
        Arg2Per,
        Arg4Per,
        GlyPer,
        PheGC,
        Leu2GC,
        Leu4GC,
        ValGC,
        Ser2GC,
        Ser4GC,
        ProGC,
        ThrGC,
        AlaGC,
        TyrGC,
        HisGC,
        GlnGC,
        AsnGC,
        LysGC,
        AspGC,
        GluGC,
        CysGC,
        Arg2GC,
        Arg4GC,
        GlyGC)
    ),
    columns=[
        "Name",
        "GCcontent",
        "IntergenomicGC",
        "GC3content",
        "PhePer",
        "Leu2Per",
        "Leu4Per",
        "IlePer",
        "MetPer",
        "ValPer",
        "Ser2Per",
        "Ser4Per",
        "ProPer",
        "ThrPer",
        "AlaPer",
        "TyrPer",
        "HisPer",
        "GlnPer",
        "AsnPer",
        "LysPer",
        "AspPer",
        "GluPer",
        "CysPer",
        "TrpPer",
        "Arg2Per",
        "Arg4Per",
        "GlyPer",
        "PheGC",
        "Leu2GC",
        "Leu4GC",
        "ValGC"
        "Ser2GC",
        "Ser4GC",
        "ProGC",
        "ThrGC",
        "AlaGC",
        "TyrGC",
        "HisGC",
        "GlnGC",
        "AsnGC",
        "LysGC",
        "AspGC",
        "GluGC",
        "CysGC",
        "Arg2GC",
        "Arg4GC",
        "GlyGC"
    ],
    index=range(1, files_tot+1)
)

folder = "results/"
print(f"Saving data to the {folder} folder")
df.to_csv(f"{folder}data.csv", index=False)

Saving data to the results/ folder
